In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [2]:
# import env file for hostname, username, password, and db_name
from env import host, user, password, db_name

In [3]:
# Pass env file authentication to container 'url'
url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [4]:
# define sql search for all records from all tables in Zillow database
sql = """
SELECT prop.*,
       pred.logerror, 
       pred.transactiondate, 
       air.airconditioningdesc, 
       arch.architecturalstyledesc, 
       build.buildingclassdesc, 
       heat.heatingorsystemdesc, 
       landuse.propertylandusedesc, 
       story.storydesc, 
       construct.typeconstructiondesc 
FROM   properties_2017 prop  
JOIN (SELECT parcelid,
				  logerror,
				  Max(transactiondate) transactiondate 
		   FROM   predictions_2017 
		   GROUP  BY parcelid, logerror) pred
	   USING (parcelid)
JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
LEFT JOIN storytype story USING (storytypeid) 
LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
WHERE  prop.latitude IS NOT NULL 
AND prop.longitude IS NOT NULL
AND transactiondate < '2018-01-01' 
AND propertylandusetypeid = 261 
"""

In [5]:
# load zillow data from saved csv or pull from sql server and save to csv
import os
file = 'zillow_data.csv'
if os.path.isfile(file):
    df = pd.read_csv(file, index_col=0)
else:
    df = pd.read_sql(sql,url)
    df.to_csv(file)

In [6]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,781532,12095076,1.0,NaN,NaN,3.0,4.0,NaN,9.0,3.0,...,6.037461e+13,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [7]:
# df_3 shape
df.shape

(52438, 68)

In [8]:
df.parcelid.duplicated().sum()

118

In [9]:
# Sort df by parcelid and then by transaction date to group by parcelid and to make sure transaction date is last
df = df.sort_values(by=['parcelid','transactiondate'])
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
37223,1087254,10711855,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,...,6.037113e+13,-0.007357,2017-07-07,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
48246,1072280,10711877,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,0.021066,2017-08-29,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
15536,1340933,10711888,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,0.077174,2017-04-04,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
12106,1878109,10711910,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,...,6.037113e+13,-0.041238,2017-03-17,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
13666,2190858,10711923,NaN,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,-0.009496,2017-03-24,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [10]:
df[df.duplicated(subset='parcelid', keep=False)].head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
9384,16179,10722858,NaN,NaN,NaN,2.0,4.0,NaN,6.0,2.0,...,6.037135e+13,0.095171,2017-03-02,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
9385,16179,10722858,NaN,NaN,NaN,2.0,4.0,NaN,6.0,2.0,...,6.037135e+13,-0.172843,2017-07-28,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
9306,1836115,10732347,NaN,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037137e+13,0.077198,2017-03-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
9307,1836115,10732347,NaN,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037137e+13,-0.221145,2017-07-25,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
1940,2119208,10739478,1.0,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037800e+13,0.083280,2017-01-13,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [11]:
df = df.drop_duplicates(subset='parcelid', keep='last')

In [12]:
df.parcelid.duplicated().sum()

0

In [13]:
df.shape

(52320, 68)

In [14]:
# Replace blank values with NaN
df = df.replace('',np.nan)

In [15]:
# check if long/lat columns have nulls
df.longitude.isnull().sum(), df.latitude.isnull().sum()

(0, 0)

***

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [16]:
# Describe without scientific notation
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror,buildingclassdesc
count,52320,52320,13615,70,47,52320,52320,0,33655,52185,...,81,52238,52319,52320,52319,52316,2068,52199,52320,0
mean,1.49691e+06,1.29968e+07,2.43959,7.1,678.979,2.30001,3.30076,nan,6.26489,2.30527,...,1,196636,529824,2016,333492,6454.74,14.1011,6.05024e+13,0.0181378,nan
std,859433,3.35092e+06,3.84793,2.66567,711.825,1.02281,0.947555,nan,1.71635,1.01807,...,0,254286,751830,0,570511,8752.48,2.4004,1.86113e+12,0.176903,nan
min,349,1.07119e+07,1,2,38,0,0,nan,1,1,...,1,129,1000,2016,161,49.18,4,6.0371e+13,-4.65542,nan
25%,757614,1.15102e+07,1,7,263.5,2,3,nan,5,2,...,1,77159,194033,2016,76194,2660.98,14,6.0374e+13,-0.0247003,nan
50%,1.50013e+06,1.25783e+07,1,7,512,2,3,nan,6,2,...,1,131905,374006,2016,218079,4650.57,15,6.03762e+13,0.00694008,nan
75%,2.24133e+06,1.41304e+07,1,7,809.5,3,4,nan,8,3,...,1,226453,619354,2016,408777,7379.27,15,6.05904e+13,0.0406021,nan
max,2.98227e+06,1.67688e+08,13,21,3560,18,14,nan,12,18,...,1,9.1649e+06,4.90612e+07,2016,4.89522e+07,586639,99,4.8303e+14,5.263,nan


In [17]:
df.shape

(52320, 68)

In [18]:
# df.hist()
# plt.tight_layout();

In [19]:
# for col in df.columns:
#     df[col].hist()

In [20]:
# for col in df.columns:
#     print(f'_-------------- {col} ---------------')
#     print(df[col].value_counts())

In [21]:
#df.info(verbose=True)

***

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [22]:
# show nulls per column
df.isna().sum()

id                              0
parcelid                        0
airconditioningtypeid       38705
architecturalstyletypeid    52250
basementsqft                52273
                            ...  
buildingclassdesc           52320
heatingorsystemdesc         18470
propertylandusedesc             0
storydesc                   52273
typeconstructiondesc        52244
Length: 68, dtype: int64

In [23]:
# show average nulls per column
df.isna().mean()

id                          0.000000
parcelid                    0.000000
airconditioningtypeid       0.739774
architecturalstyletypeid    0.998662
basementsqft                0.999102
                              ...   
buildingclassdesc           1.000000
heatingorsystemdesc         0.353020
propertylandusedesc         0.000000
storydesc                   0.999102
typeconstructiondesc        0.998547
Length: 68, dtype: float64

In [24]:
# create dataframe that has column name as first column
col_null = pd.DataFrame()
col_null['columns_name'] = df.isna().sum().index
col_null

,columns_name
0,id
1,parcelid
2,airconditioningtypeid
3,architecturalstyletypeid
4,basementsqft
...,...
63,buildingclassdesc
64,heatingorsystemdesc
65,propertylandusedesc
66,storydesc


In [25]:
# create new column that hold the sum of nulls from each column
col_null['row_null_count'] = df.isna().sum().values
col_null

,columns_name,row_null_count
0,id,0
1,parcelid,0
2,airconditioningtypeid,38705
3,architecturalstyletypeid,52250
4,basementsqft,52273
...,...,...
63,buildingclassdesc,52320
64,heatingorsystemdesc,18470
65,propertylandusedesc,0
66,storydesc,52273


In [26]:
# create new column that hold the average of nulls from each column
col_null['row_null_percent'] = df.isna().mean().values
col_null

,columns_name,row_null_count,row_null_percent
0,id,0,0.000000
1,parcelid,0,0.000000
2,airconditioningtypeid,38705,0.739774
3,architecturalstyletypeid,52250,0.998662
4,basementsqft,52273,0.999102
...,...,...,...
63,buildingclassdesc,52320,1.000000
64,heatingorsystemdesc,18470,0.353020
65,propertylandusedesc,0,0.000000
66,storydesc,52273,0.999102


1. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [27]:
# shows column nulls in each row
dft = pd.DataFrame(df.isnull().sum(axis=1), columns=['col_nulls'])
dft

,col_nulls
37223,31
48246,29
15536,31
12106,31
13666,31
...,...
42306,35
31148,36
23911,36
9001,48


In [28]:
# shows index as id
dft = dft.reset_index()
dft

,index,col_nulls
0,37223,31
1,48246,29
2,15536,31
3,12106,31
4,13666,31
...,...,...
52315,42306,35
52316,31148,36
52317,23911,36
52318,9001,48


In [29]:
asdfasdfasdf

NameError: name 'asdfasdfasdf' is not defined

In [30]:
df2 = pd.DataFrame(df.isnull().sum(axis =1), columns = ['num_cols_missing']).reset_index()\
.groupby('num_cols_missing').count().reset_index().\
rename(columns = {'index': 'num_rows' })
df2['pct_cols_missing'] = df2.num_cols_missing/df.shape[1]

In [31]:
df2

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.338235
1,24,12,0.352941
2,25,11,0.367647
3,26,30,0.382353
4,27,177,0.397059
5,28,390,0.411765
6,29,2527,0.426471
7,30,2199,0.441176
8,31,5989,0.455882
9,32,8885,0.470588


In [32]:
dft.col_nulls.value_counts()

33    11967
34    11158
32     8885
31     5989
36     4138
35     3469
29     2527
30     2199
37     1020
28      390
38      223
27      177
44       46
26       30
39       29
40       15
24       12
25       11
43       10
42        7
45        6
41        6
23        2
48        2
46        1
47        1
Name: col_nulls, dtype: int64

In [33]:
# groupsby null count to display how many rows have the same number of col NaNs
dft = dft.groupby('col_nulls').count()
dft

,index
col_nulls,
23,2
24,12
25,11
26,30
27,177
28,390
29,2527
30,2199
31,5989


In [34]:
dft = dft.reset_index()
dft

,col_nulls,index
0,23,2
1,24,12
2,25,11
3,26,30
4,27,177
5,28,390
6,29,2527
7,30,2199
8,31,5989
9,32,8885


In [35]:
dft.rename(columns = {'index': 'num_row_missing_cols' }, inplace=True)
dft

,col_nulls,num_row_cols_missing
0,23,2
1,24,12
2,25,11
3,26,30
4,27,177
5,28,390
6,29,2527
7,30,2199
8,31,5989
9,32,8885


In [36]:
dft['col_null_percent'] = df.isnull().mean(axis=1)
dft

,col_nulls,num_row_cols_missing,col_null_percent
0,23,2,0.529412
1,24,12,0.485294
2,25,11,0.500000
3,26,30,0.470588
4,27,177,0.426471
5,28,390,0.514706
6,29,2527,0.500000
7,30,2199,0.426471
8,31,5989,0.470588
9,32,8885,0.485294


***

In [37]:
# show the number of columns missing from each row
df.isna().sum(axis=1)

37223    31
48246    29
15536    31
12106    31
13666    31
         ..
42306    35
31148    36
23911    36
9001     48
28113    48
Length: 52320, dtype: int64

In [38]:
# show the percent of columns missing from each row
df.isna().mean(axis=1)

37223    0.455882
48246    0.426471
15536    0.455882
12106    0.455882
13666    0.455882
           ...   
42306    0.514706
31148    0.529412
23911    0.529412
9001     0.705882
28113    0.705882
Length: 52320, dtype: float64

In [39]:
# show the amount of rows missing specific number of column
df.isna().sum(axis=1).value_counts()

33    11967
34    11158
32     8885
31     5989
36     4138
35     3469
29     2527
30     2199
37     1020
28      390
38      223
27      177
44       46
26       30
39       29
40       15
24       12
25       11
43       10
42        7
45        6
41        6
23        2
48        2
46        1
47        1
dtype: int64

In [ ]:
row_nulls = pd.DataFrame()
row_nulls['num_cols_missing'] = df.isna().sum().values
row_nulls

In [ ]:
row_nulls['percent_cols_missing'] = df.isna().mean().values
row_nulls

In [ ]:
df.isna().sum(axis=1).value_counts().values

In [ ]:
df.isna().sum(axis=1).value_counts()

In [ ]:
row_nulls['num_row'] = df.isna().sum(axis=1).value_counts().values
row_nulls

In [ ]:
row_nulls = pd.DataFrame()
row_nulls['num_cols_missing'] = df.isna().sum().values
row_nulls['percent_cols_missing'] = df.isna().mean().values
row_nulls['num_row'] = df.isna().sum(axis=1).value_counts().values
row_nulls

In [ ]:
# transpose to make each row an attribute name
df = df.T
df.head()

In [ ]:
#df.info(verbose=True)

In [ ]:
# Add new column to of NaN sums to column position 0
df.insert(0, 'null_count', df.isnull().sum(axis=1))

In [ ]:
df.head()

In [ ]:
# Make second column as percent of total rows that have missing values for that attribute.
df.insert(1, 'null_percent', round((df.isnull().sum(axis=1) / df.shape[1])*100))
df.head()

In [ ]:
# show number of columsn with percent of nulls
df.null_percent.value_counts()

### Notes:
- 14 columsn have all nulls

***

### 1. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

***